In [1]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np

from collections import defaultdict
from collections import Counter


In [2]:
def stripSpecialChar(text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

def preProcess(text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        text = text.lower()                                     # convert all text to lower case
        text_tokens = word_tokenize(text)                       # tokenizing the text

        # stemmedWords = list([stemmer.stem(word) for word in text_tokens])
        # validTokens = [i for i in stemmedWords if i not in stopWords]

        validTokens = [i for i in text_tokens if i not in stopWords]    # removing stop words

        validTokens = [stripSpecialChar(x) for x in validTokens]   # stripping special characters
        validTokens = [x for x in validTokens if len(x) > 1]    # Choosing only words which has length > 1
        return set(validTokens)
    


In [3]:
def findJaccardCoefficient(query, filePaths):
    # query is coming preprocessed
    for i,filePath in enumerate(tqdm(filePaths)):
        try:
            file = open(filePath, encoding="utf8")
            read = file.read().replace('\n', ' ')    
        except Exception as e:
            file = open(filePath, encoding="unicode_escape")
            read = file.read().replace('\n', ' ')
        file.close()
        # this step can be done faster if we
        # store all the prePreprocssed documents in a file
        
        
        processedDoc = preProcess(read)                    
        # Finding Intersection and Union of Doc,Query
        DocInterQ  = set.intersection(processedDoc, query)
        DocUnionQ  = set.union(processedDoc, query)
        
        coeff = len(DocInterQ)/len(DocUnionQ)    #Jaccard Coefficient value
        db[filePath] = coeff                     #Dict. with key as filename and value as jaccard coeff

In [4]:
allFiles = os.walk("../Dataset/stories")
filePaths = []
for i in allFiles:
    for j in i[2]:
        filePath = i[0] + "/" + j
        filePaths.append(filePath)

db = {}        
sentence_query = input("Enter the query: ")
query = preProcess(sentence_query)      # Query Processing
findJaccardCoefficient(query, filePaths)

# Printing the top 5 documents
k = Counter(db)
topFiveDoc = k.most_common(5) 
print(" Jaccard Coefficient :    Document")  
for i in topFiveDoc:
    print("{}    :{}".format(i[1],i[0]))


Enter the query: lion cat hero



 Jaccard Coefficient :    Document
0.00819672131147541    :stories/lionwar.txt
0.007751937984496124    :stories/lionmosq.txt
0.007462686567164179    :stories/mouslion.txt
0.006802721088435374    :stories/pussboot.txt
0.006493506493506494    :stories/burltrs
